In [ ]:
%pip install --quiet --upgrade diffusers transformers accelerate invisible_watermark mediapy

In [ ]:
import mediapy as media
import random
import sys
import torch
import time
import threading
from diffusers import DiffusionPipeline

pipes = []
generators = []

for i in range(2):
    # Create a pipeline for each GPU
    pipe = DiffusionPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        torch_dtype=torch.float16,
        use_safetensors=True,
        variant="fp16",
    )
    pipes.append(pipe.to(f"cuda:{i}"))
    generator = torch.Generator(device=f"cuda:{i}").manual_seed(seed)
    generators.append(generator)

images = []

def generate_image(index):
    prompt = "Pikachu coming out of a Pokeball"
    seed = random.randint(0, sys.maxsize)

    negative_prompt = "3d, cartoon, anime, (deformed eyes, nose, ears, nose), bad anatomy, ugly"

    time0 = time.time()

    images.append(pipes[index](
        prompt=prompt,
        negative_prompt=negative_prompt,
        output_type="latent" if use_refiner else "pil",
        generator=torch.Generator(f"cuda:{index}").manual_seed(seed),
    ).images)

In [ ]:
threads = []
for i in range(2):
    thread = threading.Thread(target=generate_image, args=(i,))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()

In [ ]:
for image in images:
    media.show_images(image)